# 设置数据文件存放根目录（不直接放在该项目中，该项目有github仓库，避免大内存数据影响代码推送）

In [ ]:
boot_file_adr = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST"

# 建立映射表

In [ ]:
import pandas as pd
import os

# ========================
# 路径配置
# ========================
# boot_file_adr = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST"
raw_csv_dir = os.path.join(boot_file_adr, "raw_csv")
save_dir = os.path.join(boot_file_adr, "mapping_table")
os.makedirs(save_dir, exist_ok=True)
save_path = os.path.join(save_dir, "KAIST_vibration_mapping.csv")

# ========================
# 获取文件列表
# ========================
csv_files = sorted([f for f in os.listdir(raw_csv_dir) if f.endswith(".csv")])

# ========================
# 标签映射（Normal=1，不同转速/故障独立编号）
# ========================
mapping_rules = {
    "Normal": 1,
    "BPFI_03": 2,
    "BPFI_10": 3,
    "BPFI_30": 4,
    "BPFO_03": 5,
    "BPFO_10": 6,
    "BPFO_30": 7,
    "Misalign_01": 8,
    "Misalign_03": 9,
    "Misalign_05": 10,
    "Unbalance_0583mg": 11,
    "Unbalance_1169mg": 12,
    "Unbalance_1751mg": 13,
    "Unbalance_2239mg": 14,
    "Unbalance_3318mg": 15,
}

# ========================
# 生成映射表
# ========================
records = []
for f in csv_files:
    matched_label = None
    for key, label in mapping_rules.items():
        if key in f:
            matched_label = label
            break
    if matched_label is None:
        matched_label = -1
    records.append({
        "filename": f,
        "label": matched_label
    })

df = pd.DataFrame(records)
df.to_csv(save_path, index=False)

print(f"✅ 已生成映射表：{save_path}")
display(df)


# 数据混合

## 全类别混合，用于基本模型训练评估

### mixed data->csv

In [ ]:
import os
import pandas as pd

# ===================== 1️⃣ 参数设置 =====================
raw_dir = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/raw_csv"
mapping_path = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mapping_table/KAIST_vibration_mapping.csv"
save_path = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/mixed_allclass.csv"
used_index_path = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/used_indices.csv"

# ===================== 2️⃣ 读取映射表 =====================
mapping = pd.read_csv(mapping_path)
print(f"✅ 映射表读取成功，共 {len(mapping)} 条")

# ===================== 3️⃣ 每类真实抽取数量 =====================
sample_dict = { 1: 2000, # normal 
                2: 1000, 
                3: 800,
                4: 900, 
                
                1: 1000,
                5: 500, 
                6: 1500, 
                7: 800,

                1: 2000, 
                8: 900, 
                9: 1500,

                1: 1600, 
                10: 800, 
                11: 900, 
                12: 600, 
                
                1: 800, 
                13: 600, 
                14: 900, 
                }

# ===================== 4️⃣ 已用索引加载 =====================
if os.path.exists(used_index_path):
    used_indices = pd.read_csv(used_index_path)
else:
    used_indices = pd.DataFrame(columns=["source_file", "index"])

# ===================== 5️⃣ 创建输出文件（第一次写入时加表头） =====================
first_write = True
if os.path.exists(save_path):
    os.remove(save_path)  # 避免旧文件干扰
    print("⚙️ 已清空旧的混合文件。")

# ===================== 6️⃣ 分批抽样逻辑 =====================
for fname in os.listdir(raw_dir):
    if not fname.endswith(".csv"): 
        continue
    if "2Nm" in fname:  # 🚫 排除2Nm
        continue

    file_path = os.path.join(raw_dir, fname)
    df = pd.read_csv(file_path)
    
    # 从映射表中获取类别
    label = mapping.loc[mapping["filename"] == fname, "label"].values[0]
    df["label"] = label
    df["source_file"] = fname

    # 查出这个 label 需要抽取多少样本
    if label not in sample_dict:
        print(f"⚠️ {fname} (label={label}) 未在 sample_dict 中，跳过。")
        continue
    need_n = sample_dict[label]
    
    # 从当前文件中过滤掉已使用过的数据
    used_idx = used_indices.loc[used_indices["source_file"] == fname, "index"].tolist()
    df_avail = df.loc[~df.index.isin(used_idx)]

    # 如果当前类别的数据已经够了，跳过
    already_have = used_indices["source_file"].isin(mapping.loc[mapping["label"] == label, "filename"]).sum()
    if already_have >= need_n:
        print(f"✅ 类别 {label} 已抽取足够样本 ({already_have} >= {need_n})，跳过 {fname}")
        continue

    # 当前还需的数量
    remain = need_n - already_have
    if remain <= 0:
        continue

    # 当前文件可用数量
    take_n = min(remain, len(df_avail))
    if take_n == 0:
        continue

    sampled_df = df_avail.sample(n=take_n, random_state=42)
    
    # 写入结果文件（追加模式）
    sampled_df.to_csv(save_path, mode="a", header=first_write, index=False)
    first_write = False

    # 更新 used_indices
    new_used = pd.DataFrame({
        "source_file": fname,
        "index": sampled_df.index
    })
    used_indices = pd.concat([used_indices, new_used], ignore_index=True)

    print(f"📦 已从 {fname} (label={label}) 抽取 {take_n} 条，剩余 {remain - take_n} 条待补。")

# ===================== 7️⃣ 保存使用记录 =====================
used_indices.to_csv(used_index_path, index=False)
print(f"✅ 混合数据逐步保存完成：{save_path}")
print(f"📁 已更新使用记录：{used_index_path}")


### mixed_data->pyg

#### 时间邻点建图

In [1]:
# ================== 1️⃣ 导入路径与函数 ==================
%run ../_init_path.py
from build_pyg_data import build_local_temporal_graph
import pandas as pd
import os

# ================== 2️⃣ 参数设置 ==================
csv_path = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/mixed_allclass.csv"
save_dir = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/pyg_data/mixed_allclass"
num_edges = 10  # 每个节点的边数（上下平均分配）

# ================== 3️⃣ 预处理：忽略最后一列 ==================
if not os.path.exists(csv_path):
    raise FileNotFoundError(f"❌ 找不到混合文件: {csv_path}")

df = pd.read_csv(csv_path)
print(f"📊 原始数据形状: {df.shape}")

# 去掉最后一列（通常为标签）
df = df.iloc[:, :-1]
print(f"🧹 已移除最后一列后形状: {df.shape}")

# 保存临时文件供建图使用
temp_csv_path = csv_path.replace(".csv", "_nofinal.csv")
df.to_csv(temp_csv_path, index=False)
print(f"✅ 已生成去除标签列的临时文件: {temp_csv_path}")

# ================== 4️⃣ 调用函数建图 ==================
nodes_csv, edges_csv, graph_pt = build_local_temporal_graph(
    csv_path=temp_csv_path,
    save_dir=save_dir,
    num_edges=num_edges
)

# ================== 5️⃣ 输出结果路径 ==================
print("\n✅ 局部时序图已构建完成：")
print(f"📁 节点文件: {nodes_csv}")
print(f"📁 边文件:   {edges_csv}")
print(f"📁 图文件:   {graph_pt}")





✅ 已添加路径：/home/charles/HZU/Industrial_Software_Testing/Industrial_Software_Testing/my_CIL_V1/data_process/KAIST
✅ 已添加 my_lib：/home/charles/HZU/Industrial_Software_Testing/Industrial_Software_Testing/my_CIL_V1/data_process/KAIST/my_lib
📊 原始数据形状: (12500, 7)
🧹 已移除最后一列后形状: (12500, 6)
✅ 已生成去除标签列的临时文件: /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/mixed_allclass_nofinal.csv
✅ 图结构构建完成，共 12500 个节点，125000 条无向边
📁 nodes.csv: /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/pyg_data/mixed_allclass/nodes.csv
📁 edges.csv: /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/pyg_data/mixed_allclass/edges.csv
📁 graph.pt : /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/pyg_data/mixed_allclass/graph.pt

✅ 局部时序图已构建完成：
📁 节点文件: /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/pyg_data/mixed_allclass/nodes.csv
📁 边文件:   /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/pyg_data/mixed_allclass/edges.csv
📁 图文件:   /home/cha

#### 余弦相似度建图